In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import datetime
import time
import csv
import os

In [2]:
url = 'https://www.profesia.sk/praca/bratislava/?radius=radius30&search_anywhere=python'


html_page = requests.get(url)

soup = BeautifulSoup(html_page.content, 'html.parser')


**1st Job on 1st Page**

In [3]:
jobs = soup.find_all('ul', class_='list')

In [4]:
job = soup.find_all('li', class_='list-row')[0]

In [5]:
job

<li class="list-row">
<a class="offer-company-logo-link" href="/praca/sourcefirst-international/C115989"> <picture><img alt="Logo SourceFirst International s.r.o." class="toplogo" data-dimension8="Logo" height="0" src="https://profesiastatic.sk/images/top_logo/115989_Screenshot_2023_10_24_114005.png" type="image/png" width="0"/></picture></a><h2><a href="/praca/sourcefirst-international/O4759912?search_id=9e71b100-8096-4fae-b25f-d63dbc8f87a5" id="offer4759912"><span class="title">Python Developer</span></a></h2><span class="employer">SourceFirst International s.r.o.</span><span class="job-location" title="Bratislava">Bratislava</span><span class="label-group"><a class="half-margin-on-right" data-dimension7="Salary label" href="/praca/sourcefirst-international/O4759912?search_id=9e71b100-8096-4fae-b25f-d63dbc8f87a5">
<span class="label label-bordered green half-margin-on-top">
<svg class="icon money green" viewbox="0 0 20 20"><use xlink:href="/images/svg/money.svg#Layer_1"></use></svg>


In [6]:
name_of_job = job.find('span', class_='title').text.strip()

In [7]:
company_name = job.find('span', class_='employer').text.strip()

In [8]:
location = job.find('span', class_='job-location').text.strip()

In [9]:
salary = job.find('span', class_='label label-bordered green half-margin-on-top').text.strip()

In [10]:
added_or_updated = job.find('span', class_='info').text.strip()

In [11]:
titles = ['Name of Job', 'Company Name', 'Location', 'Salary']
data = [name_of_job, company_name, location, salary]
df = pd.DataFrame(columns=titles)

In [12]:
df.loc[len(df)] = data

In [13]:
df

,Name of Job,Company Name,Location,Salary
0,Python Developer,SourceFirst International s.r.o.,Bratislava,Od 2 800 EUR/mesiac


**1st Full Page**

In [14]:
url = 'https://www.profesia.sk/praca/bratislava/?radius=radius30&search_anywhere=python&page_num=1'

html_page = requests.get(url)

soup = BeautifulSoup(html_page.content, 'html.parser')

jobs = soup.find_all('ul', class_='list')

titles = ['Name of Job', 'Company Name', 'Location', 'Salary', 'Added or Updated', 'Date']
df_all_page = pd.DataFrame(columns=titles)

for job in jobs:
    job_listings = job.find_all('li', class_='list-row')
    for listing in job_listings:
        added_or_updated_element = listing.find('span', class_='info')
        if added_or_updated_element and ('hodinami' or 'hodinou') in added_or_updated_element.text:
            name_of_job_element = listing.find('span', class_='title')
            company_name_element = listing.find('span', class_='employer')
            location_element = listing.find('span', class_='job-location')
            salary_element = listing.find('span', class_='label label-bordered green half-margin-on-top')
            if name_of_job_element and company_name_element and location_element and salary_element:
                name_of_job = name_of_job_element.text.strip()
                company_name = company_name_element.text.strip()
                location = location_element.text.strip()
                salary = salary_element.text.strip()
                added_or_updated = added_or_updated_element.text.strip()
                today = datetime.date.today()
                data = [name_of_job, company_name, location, salary, added_or_updated, today]
                df_all_page.loc[len(df_all_page)] = data



In [15]:
df_all_page

,Name of Job,Company Name,Location,Salary,Added or Updated,Date
0,Python Developer,SourceFirst International s.r.o.,Bratislava,Od 2 800 EUR/mesiac,Pridané pred 9 hodinami,2024-02-12
1,"SW Automation Tester (Javascript/Typescript, P...",SourceFirst International s.r.o.,"Bratislava, Slovensko (Pozícia umožňuje občasn...",1 700 - 3 000 EUR/mesiac,Pridané pred 9 hodinami,2024-02-12
2,Software Developer C++ & Python (All Genders),Allplan Slovensko s. r. o.,"Bajkalská, Bratislava, Slovensko (Job with occ...",1 900 EUR/mesiac,Pridané pred 4 hodinami,2024-02-12
3,"Senior Data Engineer (Big Data, Python, Spark)",SourceFirst International s.r.o.,"Bratislava, Slovakia (Job with occasional home...",3 300 - 4 000 EUR/mesiac,Pridané pred 9 hodinami,2024-02-12
4,"Senior Data Engineer (Big Data, Python, Spark)",SourceFirst International s.r.o.,"Bratislava, Slovakia (Job with occasional home...",3 300 - 4 000 EUR/mesiac,Pridané pred 9 hodinami,2024-02-12
5,"Data Engineer (Python, PySpark, Tableau, Palan...",SourceFirst International s.r.o.,"Bratislava, Slovakia (Job with occasional home...",2 600 - 3 000 EUR/mesiac,Pridané pred 9 hodinami,2024-02-12
6,Senior System Administrator Windows s PowerShe...,"Manuvia, a. s.",Práca z domu,2 200 EUR/mesiac,Aktualizované pred 13 hodinami,2024-02-12
7,Senior DATA Engineer,SEITEQ s.r.o.,Remote work,3 100 EUR/mesiac,Pridané pred 10 hodinami,2024-02-12
8,Data Scientist ref. (889291),SourceFirst International s.r.o.,Bratislava,Od 2 500 EUR/mesiac,Pridané pred 9 hodinami,2024-02-12


**All Pages**

In [16]:



def scrape_all_pages():
    titles = ['Name of Job', 'Company Name', 'Location', 'Salary', 'More Info', 'Added or Updated', 'Date']
    df_all_pages = pd.DataFrame(columns=titles)
    page_num = 1

    while True:
        url = f'https://www.profesia.sk/praca/bratislava/?radius=radius30&search_anywhere=python&page_num={page_num}'
        html_page = requests.get(url)
        soup = BeautifulSoup(html_page.content, 'html.parser')

        jobs = soup.find_all('ul', class_='list')
        if not jobs:
            break

        for job in jobs:
            job_listings = job.find_all('li', class_='list-row')
            for listing in job_listings:
                added_or_updated_element = listing.find('span', class_='info')
                if added_or_updated_element and ('hodinami' or 'hodinou') in added_or_updated_element.text:
                    name_of_job_element = listing.find('span', class_='title')
                    company_name_element = listing.find('span', class_='employer')
                    location_element = listing.find('span', class_='job-location')
                    salary_element = listing.find('span', class_='label label-bordered green half-margin-on-top')
                    more_info_element = listing.a['href']
                    if name_of_job_element and company_name_element and location_element and salary_element and more_info_element:
                        name_of_job = name_of_job_element.text.strip()
                        company_name = company_name_element.text.strip()
                        location = location_element.text.strip()
                        salary = salary_element.text.strip()
                        added_or_updated = added_or_updated_element.text.strip()
                        more_info = 'profesia.sk'+more_info_element
                        today = datetime.date.today()
                        data = [name_of_job, company_name, location, salary, more_info, added_or_updated, today]
                        df_all_pages.loc[len(df_all_pages)] = data

        page_num += 1

    return df_all_pages


In [17]:
a = scrape_all_pages()

In [18]:
a

,Name of Job,Company Name,Location,Salary,More Info,Added or Updated,Date
0,Python Developer,SourceFirst International s.r.o.,Bratislava,Od 2 800 EUR/mesiac,profesia.sk/praca/sourcefirst-international/C1...,Pridané pred 9 hodinami,2024-02-12
1,"SW Automation Tester (Javascript/Typescript, P...",SourceFirst International s.r.o.,"Bratislava, Slovensko (Pozícia umožňuje občasn...",1 700 - 3 000 EUR/mesiac,profesia.sk/praca/sourcefirst-international/C1...,Pridané pred 9 hodinami,2024-02-12
2,Software Developer C++ & Python (All Genders),Allplan Slovensko s. r. o.,"Bajkalská, Bratislava, Slovensko (Job with occ...",1 900 EUR/mesiac,profesia.sk/praca/allplan-slovensko/O4760315?s...,Pridané pred 4 hodinami,2024-02-12
3,"Senior Data Engineer (Big Data, Python, Spark)",SourceFirst International s.r.o.,"Bratislava, Slovakia (Job with occasional home...",3 300 - 4 000 EUR/mesiac,profesia.sk/praca/sourcefirst-international/C1...,Pridané pred 9 hodinami,2024-02-12
4,"Senior Data Engineer (Big Data, Python, Spark)",SourceFirst International s.r.o.,"Bratislava, Slovakia (Job with occasional home...",3 300 - 4 000 EUR/mesiac,profesia.sk/praca/sourcefirst-international/C1...,Pridané pred 9 hodinami,2024-02-12
5,"Data Engineer (Python, PySpark, Tableau, Palan...",SourceFirst International s.r.o.,"Bratislava, Slovakia (Job with occasional home...",2 600 - 3 000 EUR/mesiac,profesia.sk/praca/sourcefirst-international/C1...,Pridané pred 9 hodinami,2024-02-12
6,Senior System Administrator Windows s PowerShe...,"Manuvia, a. s.",Práca z domu,2 200 EUR/mesiac,profesia.sk/praca/manuvia/O4757710?search_id=6...,Aktualizované pred 13 hodinami,2024-02-12
7,Senior DATA Engineer,SEITEQ s.r.o.,Remote work,3 100 EUR/mesiac,profesia.sk/praca/seiteq/O4720723?search_id=61...,Pridané pred 10 hodinami,2024-02-12
8,Data Scientist ref. (889291),SourceFirst International s.r.o.,Bratislava,Od 2 500 EUR/mesiac,profesia.sk/praca/sourcefirst-international/C1...,Pridané pred 9 hodinami,2024-02-12
9,Mail System Engineer,"Websupport, s.r.o.","Karadžičova 12, Bratislava",Od 2 500 EUR/mesiac,profesia.sk/praca/websupport/O4172092?search_i...,Pridané pred 7 hodinami,2024-02-12


In [19]:
def runner():
    while(True):
        a = scrape_all_pages()
        if not os.path.isfile('profesiaScraped.csv'):
            a.to_csv('profesiaScraped.csv', mode='a', index=False, encoding='UTF8')
        else: 
            a.to_csv('profesiaScraped.csv', mode='a', header=False, index=False, encoding='UTF8')
        print("Scraped!")
        time.sleep(86400)

In [20]:
runner()

Scraped!
Scraped!


In [21]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df = pd.read_csv('profesiaScraped.csv')
df

,Name of Job,Company Name,Location,Salary,More Info,Added or Updated,Date
0,Python Developer,SourceFirst International s.r.o.,Bratislava,Od 2 800 EUR/mesiac,profesia.sk/praca/sourcefirst-international/C1...,Pridané pred 9 hodinami,2024-02-12
1,"SW Automation Tester (Javascript/Typescript, P...",SourceFirst International s.r.o.,"Bratislava, Slovensko (Pozícia umožňuje občasn...",1 700 - 3 000 EUR/mesiac,profesia.sk/praca/sourcefirst-international/C1...,Pridané pred 9 hodinami,2024-02-12
2,Software Developer C++ & Python (All Genders),Allplan Slovensko s. r. o.,"Bajkalská, Bratislava, Slovensko (Job with occ...",1 900 EUR/mesiac,profesia.sk/praca/allplan-slovensko/O4760315?s...,Pridané pred 4 hodinami,2024-02-12
3,"Senior Data Engineer (Big Data, Python, Spark)",SourceFirst International s.r.o.,"Bratislava, Slovakia (Job with occasional home...",3 300 - 4 000 EUR/mesiac,profesia.sk/praca/sourcefirst-international/C1...,Pridané pred 9 hodinami,2024-02-12
4,"Senior Data Engineer (Big Data, Python, Spark)",SourceFirst International s.r.o.,"Bratislava, Slovakia (Job with occasional home...",3 300 - 4 000 EUR/mesiac,profesia.sk/praca/sourcefirst-international/C1...,Pridané pred 9 hodinami,2024-02-12
5,"Data Engineer (Python, PySpark, Tableau, Palan...",SourceFirst International s.r.o.,"Bratislava, Slovakia (Job with occasional home...",2 600 - 3 000 EUR/mesiac,profesia.sk/praca/sourcefirst-international/C1...,Pridané pred 9 hodinami,2024-02-12
6,Senior System Administrator Windows s PowerShe...,"Manuvia, a. s.",Práca z domu,2 200 EUR/mesiac,profesia.sk/praca/manuvia/O4757710?search_id=6...,Aktualizované pred 13 hodinami,2024-02-12
7,Data Scientist ref. (889291),SourceFirst International s.r.o.,Bratislava,Od 2 500 EUR/mesiac,profesia.sk/praca/sourcefirst-international/C1...,Pridané pred 9 hodinami,2024-02-12
8,Senior DATA Engineer,SEITEQ s.r.o.,Remote work,3 100 EUR/mesiac,profesia.sk/praca/seiteq/O4720723?search_id=6e...,Pridané pred 10 hodinami,2024-02-12
9,Mail System Engineer,"Websupport, s.r.o.","Karadžičova 12, Bratislava",Od 2 500 EUR/mesiac,profesia.sk/praca/websupport/O4172092?search_i...,Pridané pred 7 hodinami,2024-02-12


In [22]:
df.columns

Index(['Name of Job', 'Company Name', 'Location', 'Salary', 'More Info',
       'Added or Updated', 'Date'],
      dtype='object')